# SoundCloud Project

# Part 1: Data Collecting

## I. Data Collecting by API

#### General: Collecting user, playlist and track data on Soundcloud
#### Notes: The assignment of tasks in each stage is different, so there will be some members who do not participate in this stage 

**Member lists**

|Full Name|MSSV|Tasks|Contribution 
| :------ | :---: | :--------- | :--------------------------
|Lâm Thị Xuân Thy|20127082|Thu thập dữ liệu users, tracks, playlists bằng API, tiền xử lý dữ liệu và phân tích câu 4 và câu 6|100%
|Đào Ngọc Hoa|20127500|Thu thập dữ liệu playlists, tracks bằng parse HTML, Tiền xử lý dữ liệu và phân tích câu 1 và câu 2|100%
|Nguyễn Phước Gia Huy|20127038|Thu thập dữ liệu users bằng parse HTML (chỉ lấy được link), Tiền xử lý và phân tích câu 7|100%
|Trần Thị Bảo Hương|20127514|Tiền xử lý và phân tích câu 5|100%
|Lương Trường Thịnh|19120315|Tiền xử lý và phân tích câu 3|100%

## Import library
#### Libraries are used in program.

In [ ]:
import requests
import csv
import pandas as pd
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import datetime
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
import urllib.robotparser # Kiểm tra file robot.txt có được phép crawl không

In [ ]:
track_lists = []

#### - In project we collect all atributes of users and playlists which are relative to users that are collected. The tracks we collected include tracks which users have in their playlists and the others.
#### - Users data will be recorded in user.csv
#### - Playlists data will be recorded in playlists.csv
#### - Tracks data will be recorded in tracks.csv

### 1. Collecting data "User"

In [ ]:
url = "https://api-v2.soundcloud.com/tracks/193781466/likers?&limit=1000&client_id=8m4K5d2x4mNmUHLhLmsGq9vxE3dDkxCm"
resp = requests.get(url)
while resp.ok == False:
    resp = requests.get(url)
resp.encoding='utf-8-sig'
content = resp.text.encode().decode('utf-8-sig')
data = json.loads(content)

In [ ]:
data = data['collection']

In [ ]:
csv_file = 'user.csv'
csv_columns = data[0].keys() 
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for i in data:
            writer.writerow(i)
except IOError:
    print("I/O error")

### 2. Collecting data "Playlists"

In [ ]:
user = pd.read_csv('user.csv')

In [ ]:
list_id = []
for index, row in user.iterrows():
    if row['playlist_count'] != 0:
        list_id.append(row['id'])

In [ ]:
dist_ = []
for i in list_id:
    url = "https://api-v2.soundcloud.com/users/" + str(i) 
    y = '/playlists_without_albums?client_id=8m4K5d2x4mNmUHLhLmsGq9vxE3dDkxCm&limit=1000'
    url = url + y
    resp = requests.get(url)
    while resp.ok == False:
        resp = requests.get(url)
    resp.encoding='utf-8-sig'
    content = resp.text.encode().decode('utf-8-sig')
    data = json.loads(content)
    dist_.append(data)

In [ ]:
list_user = []
for i in dist_:
    list_user.append(i['collection'])

In [ ]:
def list_tracks(tracks):
    list_tracks = []
    for i in tracks:
        track = {}
        if 'publisher_metadata' in i:
            temp = i['publisher_metadata']
            track.setdefault('id_song', temp['id'])
            track.setdefault('')
    return list_tracks

In [ ]:
def return_list_in_tracks(tracks):
    list_tracks = []
    for i in tracks:
        track_lists.append(i)
        list_tracks.append(i['id'])
    return list_tracks

In [ ]:
def standard_playlist_in_users(user):
    playlist = []
    for i in user:
        each = []
        id_playlist = i['id']
        name_playlist = i['title']
        tracks = return_list_in_tracks(i['tracks'])
        each.append(id_playlist)
        each.append(name_playlist) 
        each.append(tracks)
        playlist.append(each)
    return playlist
        
        

In [ ]:
pay = {}
for i in range(0 , len(list_user)):
    if list_user[i] != []:
        user_id = list_user[i][0]['user_id']
        pay.setdefault(user_id, standard_playlist_in_users(list_user[i]))

In [ ]:
results = []
for i in pay:
    for each in pay[i]:
        temp = {}
        temp.setdefault('user_id', i)
        temp.setdefault('playlist_id', each[0])
        temp.setdefault('playlist_name', each[1])
        temp.setdefault('list_tracks', str(each[2]))
        results.append(temp)

In [ ]:
csv_file = 'playlists.csv'
csv_columns = ['user_id', 'playlist_id', 'playlist_name', 'list_tracks']
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for i in results:
            writer.writerow(i)
except IOError:
    print("I/O error")

### 3. Collecting data "Tracks"

In [ ]:
csv_file = 'tracks.csv'
csv_columns = track_lists[0].keys()
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for i in track_lists:
            writer.writerow(i)
except IOError:
    print("I/O error")

# SoundCloud Project

# Part 1: Data Collecting (HTML parse is a second method to collect data)

## II. Data Collecting by HTML

#### General: Collecting user, playlist and track data on Soundcloud
#### Notes: The assignment of tasks in each stage is different, so there will be some members who do not participate in this stage 

### DYNAMIC WEB SCRAPING WITH SELENIUM

Because our group has scrapped the web by using API, in this section we will not go to so deeply into take the full information of users, tracks and playlists, using API has done so good. Therefore, by using selenium, we just take the links of users, tracks and playlists. From these links, we can take the information that we need from them eventually.

### Import libraries

In [ ]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver

In [ ]:
sets = []
playlists = []
users = []
tracks= []
tags = []

### Define the driver_path and url

Make sure that you have download the chromedriver with suitable version to run this code

In [ ]:
DRIVER_PATH = './chromedriver'
URL = "https://soundcloud.com/discover"

In [ ]:
driver = webdriver.Chrome(DRIVER_PATH)
driver.get(URL)
driver.implicitly_wait(30)
for i in range(5):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

In [ ]:
data= driver.page_source
soup = BeautifulSoup(data, "html.parser")

We can see that most of the sets are contained in class "tileGallery__sliderPanelSlide", so I will get it first of all

In [ ]:
slides = soup.find_all(class_="tileGallery__sliderPanelSlide")

### Get sets

I see that many playlists are contained in the sets, so I will get them

In [ ]:
for i in slides:
    values = i.find_all('a',href = True)
    values = [k['href'] for k in values if k]
    if values:
        for val in values:
            if 'discover' in val:
                playlists.append("https://soundcloud.com"+val)
            elif 'sets' in val:
                sets.append(val.partition('sets')[0]+'sets')
            elif val!='':
                users.append("https://soundcloud.com"+val)
sets = list(set(sets))

### Get playlists and tags from sets

When I get the playlists from the sets, I get the tags at the same time. Tags are important because they contain a lot of playlists. Just taking from sets is not enough 1000 playlists.

In [ ]:
links = ["https://soundcloud.com"+s for s in sets]

In [ ]:
for link in links:
    if len(playlists) > 1000:
        break
    driver = webdriver.Chrome(DRIVER_PATH)
    driver.get(link)
    driver.implicitly_wait(30);
    for i in range(10):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
    data= driver.page_source
    soup = BeautifulSoup(data, "html.parser")
    items = soup.find_all(class_="soundList__item")
    for d in items:
        routes = d.find_all('a',href=True)
        if routes:
            values = [k['href'] for k in routes if k]
            for val in values:
                if 'sets' in val:
                    playlists.append("https://soundcloud.com"+val)
                    print(val)
                elif 'tags' in val:
                    tags.append("https://soundcloud.com"+val+'/playlists')
    playlists = list(set(playlists))
    tags = list(set(tags))
    driver.close()

### Get playlists from tags

Because I have collected not enough playlists, I decide to collect from tags now

In [ ]:
links = tags[:]

### The agorithm is just as getting playlists from sets.

In [ ]:
for link in links:
    if len(playlists) > 1000:
        break
    driver = webdriver.Chrome(DRIVER_PATH)
    driver.get(link)
    driver.implicitly_wait(30);
    for i in range(20):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
    data= driver.page_source
    soup = BeautifulSoup(data, "html.parser")
    items = soup.find_all(class_="soundList__item")
    for d in items:
        routes = d.find_all('a',href=True)
        if routes:
            values = [k['href'] for k in routes if k]
            for val in values:
                if 'sets' in val:
                    playlists.append("https://soundcloud.com"+val)
    
    driver.close()
    playlists=list(set(playlists))

### Get tracks and users from these playlists

If we see the links of the tracks, we can see that they contains the information of their owners, so we will get the users from them.

In [ ]:
links = playlists[:]

In [ ]:
for link in links:
    if len(tracks) > 1000:
        break     
    driver = webdriver.Chrome(DRIVER_PATH)
    driver.get(link)
    driver.implicitly_wait(30);
    for i in range(10):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
    data= driver.page_source
    soup = BeautifulSoup(data, "html.parser")
    data = soup.find_all(class_ = 'trackList__item sc-border-light-bottom sc-px-2x')
    for d in data:
        val = d.find(class_='trackItem__trackTitle sc-link-dark sc-link-primary sc-font-light',href=True)
        if val:
            h_ref = val['href']
            name = val.get_text()
            tracks.append('https://soundcloud.com'+h_ref)
            users.append('https://soundcloud.com/'+h_ref.split("/", 2)[1])
            print(name)
    driver.close()
    tracks = list(set(tracks))
    users = list(set(users))

### Get users from playlists

Because the number of users is not enough 1000, so I decided to get them from playlists too. If we see the links of the playlists, we can see that they contains the information of their owners, so we will get the users from them.

In [ ]:
for playlist in playlists:
    if len(users) > 1000:
        break
    users.append('https://soundcloud.com/'+playlist.split('/')[3])
    users = list(set(users))

### Save users.csv, tracks.csv and playlist.csv

#### 1. Save tracks

In [ ]:
head = 'Link\n'
with open('tracks.csv', mode = 'w')as f:
    f.write(head)
    for track in tracks:
        f.write(track)
        f.write('\n')
f.close()

#### 2. Save users

In [ ]:
head = 'Link\n'
with open('users.csv', mode = 'w')as f:
    f.write(head)
    for user in users:
        f.write(user)
        f.write('\n')
f.close()

#### 3. Save playlists

In [ ]:
head = 'Link\n'
with open('playlists.csv', mode = 'w')as f:
    f.write(head)
    for playlist in playlists:
        f.write(playlist)
        f.write('\n')
f.close()

From these files of links, we can scrap more information of them when we need.

# SoundCloud Project

# Part 2: Data Analysis

#### General: Analysis data in user, playlist and track data on Soundcloud
#### Notes: The assignment of tasks in each stage is different, so there will be some members who do not participate in this stage 

## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## I. Track

### 1. Data Exporatory

#### Load data

In [ ]:
tracks = pd.read_csv('tracks.csv', index_col = 'id')
tracks.head(5)

#### How many rows and columns in dataset?

In [ ]:
rows_num, cols_num = tracks.shape
print(rows_num, cols_num)

#### Does data have duplicated sample?

In [ ]:
sum(tracks.duplicated()) == 0

This data have duplicated sample

#### What data type that current dataset have? Is there a column with a data type that is not suitable for further processing?

In [ ]:
tracks.dtypes

#### How many rows of each columns that have value?

In [ ]:
tracks.count()

The strange thing is that most of columns have only 3940 rows is diffent from none, while some columns like policy have up to 34685 rows. This means that it exists some rows that most of its attributes are none.

#### How many distinct values that current dataset have?

In [ ]:
distinct_values_num = tracks.nunique(axis=0)
distinct_values_num 

The strange thing is that some columns have only 1 distinct value, which has no mean for data analysis

### Data Preproccessing

#### Change the dtype of "created_at", "display_date", ".release_date", "last_modified" to datetime

In [ ]:
tracks.created_at = pd.to_datetime(tracks.created_at)
tracks.display_date= pd.to_datetime(tracks.display_date)
tracks.release_date= pd.to_datetime(tracks.release_date)
tracks.last_modified= pd.to_datetime(tracks.last_modified) 

#### Delete columns that have no or only 1 distinct values

In [ ]:
tracks = tracks.loc[:,distinct_values_num>1]

#### Drop columns that have abnormal rows number

In [ ]:
row_have_values_count = tracks.count()
tracks = tracks.loc[:,row_have_values_count<row_have_values_count.mean()]

#### Drop rows whose values are none totally

In [ ]:
tracks.dropna(how='all',inplace = True)

#### Drop duplicated samples

In [ ]:
tracks.drop_duplicates(inplace = True)

### Data Exploration (cont.)

In [ ]:
rows_num, cols_num = tracks.shape
print(rows_num, cols_num)

#### How does the numeric data distribute?

In [ ]:
# YOUR CODE HERE
data = tracks.describe()
missing_percentage = ((1-data.loc['count']/rows_num) * 100).round(1)
nume_col_info_df = pd.DataFrame({'missing_percentage':missing_percentage,
                                 'min': data.loc['min'],
                              'lower_quartile': data.loc['25%'],
                              'median': data.loc['50%'],
                                 'upper_quartile': data.loc['75%'],
                                'max': data.loc['max']})
nume_col_info_df = nume_col_info_df.T
nume_col_info_df

#### How does the categorial data distribute?

In [ ]:
data = tracks.select_dtypes(include=['object'])

diff_vals=data.stack().dropna(how='any').reset_index(level= 0,drop=True).groupby(level=0).unique().reindex(index=data.columns).rename('diff_vals')
num_diff_vals = diff_vals.str.len().rename('num_diff_vals')
missing_ratio = (1-data.count().rename('missing_ratio')/rows_num)*100

cate_col_profiles_df = pd.concat([missing_ratio,num_diff_vals,diff_vals], axis=1)
cate_col_profiles_df

#### Create the new columns, which is the number of medias that post that track

In [ ]:
import ast
tracks['medias_count'] = tracks.media.apply(lambda x: len(ast.literal_eval(x)['transcodings']))

In [ ]:
tracks.waveform_url

# Make question

## 1. What are the most trending tracks from 2020 to now in soundcloud?

A track is consider trending if many users like, download, commend, report and play back it.

In [ ]:
cols = ['comment_count','download_count','likes_count','playback_count','reposts_count','medias_count']
data = tracks.loc[tracks.created_at.dt.year >= 2020,cols]
data

## Using K-Means Agorithm

In this project, I decided to use K-Mean agorithm because it is quite easy to understand, simple to implement and it guarantees the convergence. This means that we will always find the solution for the problems.

#### Group assignment

With given initial value of centroids, we compute the distance between each centroid with each vector and assign the group according to the smallest distance. The function then returns a vector of groupings.

In [ ]:
def group_assignment(data,centroids):
    grouping_vec_c = np.zeros(len(data))
    for i in range(len(data)):
        dist = np.zeros(len(centroids))
        for j in range(len(centroids)):
            dist[j] = np.linalg.norm(data.iloc[i]-centroids.iloc[j])
        grouping_vec_c[i] = dist.argmin()
    return grouping_vec_c

In [ ]:
#TEST
centroids = data.sample(n = 3)
grouping = group_assignment(data,centroids)
grouping[:5]

#### Update centroid

We define a function to update the centroid after the group assignment, returning a new list of group centroids. The new controids are defined as the mean of their grouping points.

In [ ]:
def update_centroid(data, grouping, centroids):
    new_centroids = pd.DataFrame(columns=centroids.columns)
    for i in range(len(centroids)):
        new_centroids = new_centroids.append(data[grouping == i].mean(axis = 0), ignore_index = True)
    return new_centroids

In [ ]:
new_centroids = update_centroid(data, grouping, centroids)

## K-Means Agorithm

The agorithm is as follow:
    <ul>input: 
        <li>data: dataset that is considered</li>
        <li>centroids: the k samples that are chosen randomly from dataset</li>
    </ul>
    <ul> output:
    <li>new_centroids: is the final centroids of clusters</li>
    <li>grouping: is the group of each itemset in dataset belongs to</li>
    </ul>
    
    ```python
        while True:
            Assign group for each itemset to the centroid that is near it most.
            Find the new centroids of new clusters. The new centroids are  the mean of all itemsets in the new cluster
            Terminate the algorithm when the distance between centroids and new_centroids become so small
      ```

In [ ]:
def KMeans_agorithm(data, centroids):
    while True:
        grouping = group_assignment(data,centroids)
        new_centroids = update_centroid(data, grouping, centroids)
        if np.linalg.norm(np.array(new_centroids)-np.array(centroids)) <1e-6:
            break
        else:
            centroids = new_centroids
    return new_centroids, grouping

## Analysis
### Test the K-Means Agorithm in the dataset to find the most trending tracks from 2020 to now

In [ ]:
k = 5
centroids = data.sample(n = k)
new_centroids, grouping = KMeans_agorithm(data, centroids)

In this project, we find the trending tracks by searching the cluster whose centroid has the highest mean. After finding it, the links of these songs as follow:

In [ ]:
trending_tracks_id = data[grouping == new_centroids.mean(axis=1).argmax()].index
tracks.loc[trending_tracks_id].permalink_url

And below is the detail information of the trending tracks.

In [ ]:
a = tracks.loc[trending_tracks_id].publisher_metadata.values
info = []
for i in range(len(a)):
    info.append(eval(a[i]))
df = pd.DataFrame(info)
df = df.set_index('id')
df

## Conclusion

From above result, the Soudcloud can recommend the top trending tracks for its' homepage, or make prizes for the best tracks from 2020 to now.

## 2. How to predict the likes' number by using attributes 'comment_count', 'playback_count', 'reposts_count', 'medias_count', 'download_count'.

# Linear Regression

In this section, we will use linear regression, by using 5 attribute 'comment_count', 'playback_count', 'reposts_count', 'medias_count', 'download_count' to predict the like's number.

First of all, we set up some nessarary function.

In [ ]:
cols = ['comment_count','likes_count','playback_count','reposts_count','medias_count','download_count']
data = tracks[cols].dropna(how ='any')
X = data[['comment_count','playback_count','reposts_count','medias_count','download_count']]
Y = data.likes_count

In [ ]:
# NECESSARY FUNCTION
def fitting(A, b):
    return np.linalg.pinv(A) @ b
def RMSE(y_true,y_pred):
    return (sum((y_true-y_pred)**2)/len(y_pred))**0.5

In [ ]:
# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
trains_num = int(len(X)*2/3)
tests_num = len(X) - trains_num
X_train = X.head(trains_num).iloc[:,:]
y_train = Y[:trains_num]
X_test = X.tail(tests_num).iloc[:,:]
y_test = Y[trains_num:]

In [ ]:
#CODE
X_tmp = np.column_stack((np.ones(len(y_train)),X_train))
w = fitting(X_tmp,y_train)
XTest = np.column_stack((np.ones(len(y_test)),X_test))
y_pred = XTest @w
np.round(w,1)

#### RMSE Value

In [ ]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
RMSE(y_pred,y_test)

In [ ]:
#Biểu diễn sự chênh lệch giữa tuổi thọ trung bình dự đoán và thực tế
fig, ax = plt.subplots()
ax.scatter(y_pred, y_test, edgecolors=(0, 0, 0))
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "k--", lw=4)
ax.set_xlabel("Predicted")
ax.set_ylabel("Measured")
plt.show()

## K-folds Validation

In this section, we will use K-folds validation to find the model with the best attribute to predict the number of likes.

The agorithm is as follow:
    <ul>input: 
        <li>data: dataset that is considered</li>
        <li>centroids: the k samples that are chosen randomly from dataset</li>
    </ul>
    <ul> output:
    <li>new_centroids: is the final centroids of clusters</li>
    <li>grouping: is the group of each itemset in dataset belongs to</li>
    </ul>
    
    ```python
        while True:
            Assign group for each itemset to the centroid that is near it most.
            Find the new centroids of new clusters. The new centroids are  the mean of all itemsets in the new cluster
            Terminate the algorithm when the distance between centroids and new_centroids become so small
      ```


In [ ]:
# I=np.array_split(np.random.permutation(num_rows),k_fold)
# I[6:]

In [ ]:
#CODE
def k_folds_validation(X,y, k_fold):
    num_rows,num_cols= X.shape
    k_fold = 5
    I=np.array_split(np.random.permutation(num_rows),k_fold)
    rmse = np.zeros((num_cols,k_fold))
    for i in range(num_cols):
        X_train_feature = X.to_numpy()[:,i]
        Y_train_feature = y.to_numpy()[:]
        for k in range(4):
            Itest = I[k]
            Itrain = I.pop(k)
            X_val = np.column_stack((np.ones(len(Itest)),X_train_feature[Itest]))
            y_val = Y_train_feature[Itest]
            X_Ktrain_feature = np.column_stack((np.ones(len(Itrain)),X_train_feature[Itrain]))
            Y_Ktrain_feature = Y_train_feature[Itrain]
            w = fitting(X_Ktrain_feature,Y_Ktrain_feature)
            y_val_pred = X_val @ w
            rmse[i,k] = RMSE(y_val_pred,y_val)
            break
    return np.average(rmse,axis=1) 
rmse = k_folds_validation(X, Y, 5)
# In ra các kết quả cross-validation
df3 = pd.DataFrame({'Model with 1 attribute' : X.columns, 'RMSE': rmse })
df3

### Conclusion

From this result, we can see that the model with playback_count is the best model to predict like_count, when its RMSE is the smallest value. There fore, Soundcloud can suggest that the best way for making the tracks have the highest likes is to increase playback_count.

## User

### Data Exporatory

#### Load data

In [ ]:
users = pd.read_csv('user.csv', index_col = 'id')
users.head(5)

#### How many rows and columns in dataset?

In [ ]:
rows_num, cols_num = users.shape
print(rows_num, cols_num)

#### Does data have duplicated sample?

In [ ]:
sum(users.duplicated()) == 0

This data doesn't have duplicated sample

#### What data type that current dataset have? Is there a column with a data type that is not suitable for further processing?

In [ ]:
users.dtypes

#### How many rows of each columns that have value?

In [ ]:
users.count()

#### How many rows of each columns that have value?

In [ ]:
users.count()

There is nothing strange.

#### How many distinct values that current dataset have?

In [ ]:
distinct_values_num = users.nunique(axis=0)
distinct_values_num 

The strange thing is that some columns have only 1 distinct value, which has no mean for data analysis

### Data Preproccessing

#### Change the dtype of "created_at", "last_modified" to datetime

In [ ]:
users.created_at = pd.to_datetime(users.created_at)
users.last_modified= pd.to_datetime(users.last_modified) 

#### Delete columns that have no or only 1 distinct values

In [ ]:
users = users.loc[:,distinct_values_num>1]
users

### Data Exploration (cont.)

In [ ]:
rows_num, cols_num = users.shape
print(rows_num, cols_num)

#### How does the numeric data distribute?

In [ ]:
# YOUR CODE HERE
data = users.describe()
missing_percentage = ((1-data.loc['count']/rows_num) * 100).round(1)
nume_col_info_df = pd.DataFrame({'missing_percentage':missing_percentage,
                                 'min': data.loc['min'],
                              'lower_quartile': data.loc['25%'],
                              'median': data.loc['50%'],
                                 'upper_quartile': data.loc['75%'],
                                'max': data.loc['max']})
nume_col_info_df = nume_col_info_df.T
nume_col_info_df

#### How does the categorial data distribute?

In [ ]:
data = users.select_dtypes(include=['object'])

diff_vals=data.stack().dropna(how='any').reset_index(level= 0,drop=True).groupby(level=0).unique().reindex(index=data.columns).rename('diff_vals')
num_diff_vals = diff_vals.str.len().rename('num_diff_vals')
missing_ratio = (1-data.count().rename('missing_ratio')/rows_num)*100

cate_col_profiles_df = pd.concat([missing_ratio,num_diff_vals,diff_vals], axis=1)
cate_col_profiles_df

## 3. Who are reporters that have the most interactive in each groups from 2020 to now in soundcloud?

A reporter is consider as popular if that person do not create any track, but follow other artists, take others tracks to their playlists, have a large number of likes and followers.

In [ ]:
cols = ['followers_count','followings_count','likes_count','playlist_likes_count','playlist_count']
data = users.loc[(users.track_count == 0)  & (users.followings_count != 0) & (users.created_at.dt.year >= 2020),cols]
data

In [ ]:
list_means = data.mean(axis = 1)

This section, we will continue to use K-Mean Agorithm for clustering analysis to find the most popular reporters.

In [ ]:
k = 5
centroids = data.sample(n = k)
new_centroids, grouping = KMeans_agorithm(data, centroids)

In [ ]:
list_group = {}

In [ ]:
for i in range(len(grouping)):
    if grouping[i] not in list_group:
        list_group[grouping[i]] = {}
        list_group[grouping[i]].setdefault(i,list_means.iloc[i])
    else:
        list_group[grouping[i]].setdefault(i,list_means.iloc[i])

In [ ]:
top = {}
for i in list_group:
    max_value = max(list_group[i], key=list_group[i].get)
    top[i] = max_value

In [ ]:
ls = top.values()

In [ ]:
ls = list(ls)

In [ ]:
q = data.iloc[ls].index

In [ ]:
users.loc[q].permalink_url

As the way we have done for finding the most trending tracks, the links of these reporters as below:

### Conclusion

After this result, the SoundCloud platform can use it to recommend the playlists of these reporters as the most popular playlists in its homepage. Because the playlists of these reporters are very catching and loved by the public, which makes these reporters have a large number of followers and likes.

## 4. Which genre that users like the most?

In [ ]:
convert = tracks['genre'].values

In [ ]:
for i in range(0, len(convert)):
    if type(convert[i]) != str:
        convert[i] = 'free'

In [ ]:
tracks['genre'] = convert

In [ ]:
playlist = pd.read_csv('playlists.csv')

In [ ]:
p = []

In [ ]:
for temp in playlist['list_tracks']:
    if temp != []:
        temp =  temp.rstrip(']')
        temp =  temp.lstrip('[')
        temp = temp.split(', ')
        if '' in temp:
            temp.remove('')
        temp = [int(i) for i in temp]
        p.append(temp)
    else:
        p.append(NaN)

In [ ]:
playlist['list_tracks'] = p

In [ ]:
def return_list_tracks(users):
    list_users = {}
    for index, row in playlist.iterrows():
        if row['user_id'] not in list_users:
            list_users[row['user_id']] = row['list_tracks']
        else:
            list_users[row['user_id']].extend(row['list_tracks'])
    return list_users
        

In [ ]:
list_data= return_list_tracks(list(playlist.index))

In [ ]:
genres= {}

for user in list_data:
    genres[user] = []
    for track in list_data[user]:
        if track in tracks.index:
            genre = tracks.loc[track]['genre']
            genres[user].append(genre)
        

In [ ]:
results = {}
for user in genres:
    after = []
    for genre in genres[user]:
        if type(genre) == str:
            after.append(genre)
    u = dict([x,after.count(x)] for x in set(after))
    if u != {}:
        max_value = max(u, key=u.get)
        results[user] = max_value
    else:
        results[user] = None
    
    
        

In [ ]:
print(results)

There has users they don't store tracks in their playlists, so we can't not know what kind of genre they likes. The data of these users will be None. Others all genres of users will be stored in results variable.

## 5. Which tracks that users like the most?

Only consider on this data
We consider on the average of likes count, playback count and reports count.

In [ ]:
handle_data = tracks[['likes_count', 'playback_count', 'reposts_count']]

In [ ]:
pos = handle_data.mean(axis = 1).argmax()

In [ ]:
id_track = tracks.iloc[pos].name

The track users like the most has the id is: 

In [ ]:
id_track

Link track:

In [ ]:
print(tracks.iloc[pos].permalink_url)

## 6. Find the relationship between followers_count and following_count

In [ ]:
users_df = pd.read_csv('user.csv')
users_df.head()

In [ ]:
users_df = users_df.drop_duplicates() if (users_df.duplicated()).any() else users_df

In [ ]:
rows_users = len(users_df)
cols_users = len(users_df.columns)
print ('rows: ', rows_users, '\ncols: ',cols_users)
attribute_users = users_df.columns

In [ ]:
users_df[attribute_users].isnull().sum(axis=0)/users_df[attribute_users].isnull().count(axis=0) * 100

In [ ]:
users_df=users_df[['id','username','full_name','followers_count','followings_count','playlist_count','track_count','permalink_url']]

In [ ]:
df=users_df[:].copy()

# prepare for the linear regression
X=np.ones(shape=(df.shape[0],2))
X[:,0]=np.array(df['followings_count'])
y=np.array(df['followers_count'])

In [ ]:
# linearRegression class
class OLSLinearRegression:
    def fit(self, X, y): # calculate weights

        self.w =np.linalg.pinv(X) @ y
        return self

    def get_params(self):
        return self.w

    def predict(self, X): # predict
        return np.sum(self.w.ravel() * X, axis=1)   

# Hàm tính rmse 
def rmse(y_test,y_predict):
    return np.sqrt(np.mean((y_predict-y_test)**2))

In [ ]:
w=OLSLinearRegression().fit(X,y)
w.get_params()

In [ ]:
print('rmse=',rmse(y,w.predict(X)))

plt.figure(figsize=(10,6))

plt.title('followings_count vs followers_count')
plt.xlabel('followings_count')
plt.ylabel('followers_count')

plt.scatter(X[:,0], y, color ='b',alpha=0.3)
plt.plot(X[:,0], w.predict(X), color ='k')
plt.show()


Look at chart, we can see there are quite few popular users. There are few people who have less followings_count but have much more following_count than other people. And the rmse is 36.4627, so that users are quite fit to the linear, the number of followers is directly proportional to the number of followings. 

*followers_count = 0.16685729 \* followings_count + 2.70837616*

## 7. Who is the most popular reposter?

Take out the reposters from the data

In [ ]:
reposter_df= users_df[(users_df['track_count']==0) & (users_df['followers_count']>10) & (users_df['playlist_count']>0)].iloc[:,:]
reposter_df
# users_df['track_count']

In [ ]:

plt.figure(figsize=(10,6))

plt.title('reposters')
plt.xlabel('followers_count')
plt.ylabel('playlist_count')

plt.scatter(reposter_df['followers_count'], reposter_df['playlist_count'], color ='b',alpha=0.3)
# plt.plot(X, w.predict(X), color ='k')
plt.show()

The most popular reposter is

In [ ]:
reposter_df[reposter_df['followers_count']==max(reposter_df['followers_count'])]